<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# A Baseline Model for the Radiant Earth Spot the Crop Challenge

This notebook walks you through the steps to load the data and build a baseline model using Random Forests for `Radiant Earth Spot the Crop Challenge`.

## Radiant MLHub API


The Radiant MLHub API gives access to open Earth imagery training data for machine learning applications. You can learn more about the repository at the [Radiant MLHub site](https://mlhub.earth) and about the organization behind it at the [Radiant Earth Foundation site](https://radiant.earth).

Full documentation for the API is available at [docs.mlhub.earth](docs.mlhub.earth).

Each item in our collection is explained in json format compliant with [STAC](https://stacspec.org/) [label extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/label) definition.

## Dependencies

All the dependencies for this notebook are included in the `requirements.txt` file included in this folder.


**You must replace the `YOUR_API_KEY_HERE` text with your API key which you can obtain by creating a free account on the [MLHub Dashboard](https://dashboard.mlhub.earth/) within the `API Keys` tab at the top of the page.**

In [1]:
from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit
import pickle
import random

## Downloading and Loading the Data

In this part, we will download the data from Radiant MLHub and load the properties of each item in the dataset into a DataFrame


In [2]:
os.environ['MLHUB_API_KEY'] = 'YOUR_API_KEY_HERE'

collections = [
    'ref_south_africa_crops_competition_v1_train_labels',
    #'ref_south_africa_crops_competition_v1_train_source_s1', 
    'ref_south_africa_crops_competition_v1_test_labels',
    #'ref_south_africa_crops_competition_v1_test_source_s1', 
     'ref_south_africa_crops_competition_v1_test_source_s2' # Uncomment this out if you want to download the Sentinel-2 Data (not needed for the Hackathon)
     'ref_south_africa_crops_competition_v1_train_source_s2', # Uncomment this out if you want to download the Sentinel-2 Data (not needed for the Hackathon)

]

def download(collection_id):
    print(f'Downloading {collection_id}...')
    collection = Collection.fetch(collection_id)
    path = collection.download('.')
    path = collection_id + '.tar.gz'
    tar = tarfile.open(path, "r:gz")
    tar.extractall()
    tar.close()
    os.remove(path)
    
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
    
def load_df(collection_id):
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            link_path = resolve_path(current_path, link['href'])
            source_path = os.path.dirname(link_path)
            try:
                source_item = json.load(open(link_path, 'r'))
            except FileNotFoundError:
                continue
            datetime = source_item['properties']['datetime']
            satellite_platform = source_item['collection'].split('_')[-1]
            for asset_key, asset in source_item['assets'].items():
                rows.append([
                    tile_id,
                    datetime,
                    satellite_platform,
                    asset_key,
                    str(resolve_path(source_path, asset['href']))
                ])
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

# for c in collections:
#     download(c)

#competition_train_df = load_df('ref_south_africa_crops_competition_v1_train_labels')
#competition_test_df = load_df('ref_south_africa_crops_competition_v1_test_labels')

In [3]:
#competition_train_df

In [4]:
#create a file
#picklefile = open('competition_train_df', 'wb')
#pickle the dataframe
#pickle.dump(competition_train_df, picklefile)
#close file
#picklefile.close()

#create a file
#picklefile = open('competition_test_df', 'wb')
#pickle the dataframe
#pickle.dump(competition_test_df, picklefile)
#close file
#picklefile.close()

In [5]:
#read the pickle file
picklefile = open('competition_train_df', 'rb')
#unpickle the dataframe
competition_train_df = pickle.load(picklefile)
#close file
picklefile.close()

#print the dataframe
competition_train_df

,tile_id,datetime,satellite_platform,asset,file_path
0,2587,None,None,documentation,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
1,2587,None,None,field_ids,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2,2587,None,None,field_info_train,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
3,2587,None,None,labels,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
4,2587,None,None,raster_values,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
...,...,...,...,...,...
2039727,2198,2017-11-30T00:00:00+0000,s2,B09,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2039728,2198,2017-11-30T00:00:00+0000,s2,B11,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2039729,2198,2017-11-30T00:00:00+0000,s2,B12,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2039730,2198,2017-11-30T00:00:00+0000,s2,B8A,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...


In [6]:
#read the pickle file
picklefile = open('competition_test_df', 'rb')
#unpickle the dataframe
competition_test_df = pickle.load(picklefile)
#close file
picklefile.close()

#print the dataframe
competition_test_df

,tile_id,datetime,satellite_platform,asset,file_path
0,0590,None,None,documentation,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
1,0590,None,None,field_ids,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2,0590,None,None,field_info_test,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
3,0590,None,None,raster_values,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
4,0590,None,None,sample_submission,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
...,...,...,...,...,...
872183,0947,2017-11-30T00:00:00+0000,s2,B09,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
872184,0947,2017-11-30T00:00:00+0000,s2,B11,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
872185,0947,2017-11-30T00:00:00+0000,s2,B12,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
872186,0947,2017-11-30T00:00:00+0000,s2,B8A,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...


In [7]:
# This DataFrame lists all types of assets including documentation of the data. 
# In the following, we will use the Sentinel-1 bands (VV and VH) as well as labels. 
competition_train_df['asset'].unique()
competition_train_df['asset'].value_counts()
#competition_train_df[competition_train_df['asset']=='field_ids']

B12                 144466
B06                 144466
B08                 144466
B02                 144466
B03                 144466
B05                 144466
B01                 144466
B07                 144466
B8A                 144466
B09                 144466
B04                 144466
B11                 144466
CLM                 144466
VH                   74212
VV                   74212
labels                2650
field_ids             2650
raster_values         2650
documentation         2650
field_info_train      2650
Name: asset, dtype: int64

In [8]:
competition_train_df

,tile_id,datetime,satellite_platform,asset,file_path
0,2587,None,None,documentation,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
1,2587,None,None,field_ids,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2,2587,None,None,field_info_train,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
3,2587,None,None,labels,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
4,2587,None,None,raster_values,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
...,...,...,...,...,...
2039727,2198,2017-11-30T00:00:00+0000,s2,B09,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2039728,2198,2017-11-30T00:00:00+0000,s2,B11,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2039729,2198,2017-11-30T00:00:00+0000,s2,B12,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...
2039730,2198,2017-11-30T00:00:00+0000,s2,B8A,C:\apps\personal\DESARROLLO\workspace_GIT\gis\...


In [9]:
tmp = competition_train_df.groupby(['tile_id','asset']).count().reset_index()
tmp[tmp['asset']=='field_ids']

,tile_id,asset,datetime,satellite_platform,file_path
16,0001,field_ids,0,0,1
36,0002,field_ids,0,0,1
56,0003,field_ids,0,0,1
76,0004,field_ids,0,0,1
96,0005,field_ids,0,0,1
...,...,...,...,...,...
52916,2646,field_ids,0,0,1
52936,2647,field_ids,0,0,1
52956,2648,field_ids,0,0,1
52976,2649,field_ids,0,0,1


In [10]:
tile_ids_train = competition_train_df['tile_id'].unique()
tile_ids_train

array(['2587', '1302', '1130', ..., '0099', '1379', '2198'], dtype=object)

In [11]:
# For simplicty of this baseline model, we will use only 5 observations throughout the growing season
# You can choose to use all of them, select a few of them at specifc intervals, or 
# load as many as you want and interpolate between them to have a regular temporal frequency.
n_obs = 5

In [12]:
bands=['B01', 'B02', 'B03', 'B04', 'B05','B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'CLM']

In [13]:
df_s2 = competition_train_df[competition_train_df['satellite_platform']=='s2']

In [14]:
tmp = df_s2.groupby(['tile_id','asset']).count().reset_index()
for band in bands:
    print(len(tmp[tmp['asset']==band]['datetime']))
tmp

2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650
2650


,tile_id,asset,datetime,satellite_platform,file_path
0,0001,B01,76,76,76
1,0001,B02,76,76,76
2,0001,B03,76,76,76
3,0001,B04,76,76,76
4,0001,B05,76,76,76
...,...,...,...,...,...
34445,2650,B09,38,38,38
34446,2650,B11,38,38,38
34447,2650,B12,38,38,38
34448,2650,B8A,38,38,38


In [15]:
df_s2 = competition_train_df[competition_train_df['satellite_platform']=='s2']
tmp= df_s2.groupby(['tile_id','asset']).count().reset_index()
tmp['datetime'].value_counts()

38    18811
76    14469
47      169
40      130
68      130
51       91
55       78
75       78
67       65
54       52
50       52
39       39
42       39
48       39
59       26
58       26
49       26
56       26
60       13
65       13
74       13
73       13
41       13
70       13
37       13
62       13
Name: datetime, dtype: int64

In [16]:
for value in tmp['datetime'].value_counts().index.tolist():
    tile_ids_train=  tmp[tmp['datetime']==value]['tile_id'].unique()
    print(value,len(tile_ids_train)/2650)

38 0.5460377358490566
76 0.42
47 0.004905660377358491
40 0.0037735849056603774
68 0.0037735849056603774
51 0.0026415094339622643
55 0.0022641509433962265
75 0.0022641509433962265
67 0.0018867924528301887
54 0.0015094339622641509
50 0.0015094339622641509
39 0.0011320754716981133
42 0.0011320754716981133
48 0.0011320754716981133
59 0.0007547169811320754
58 0.0007547169811320754
49 0.0007547169811320754
56 0.0007547169811320754
60 0.0003773584905660377
65 0.0003773584905660377
74 0.0003773584905660377
73 0.0003773584905660377
41 0.0003773584905660377
70 0.0003773584905660377
37 0.0003773584905660377
62 0.0003773584905660377


In [17]:
tile_ids_train=  tmp[tmp['datetime']==38]['tile_id'].unique()
tile_ids_train

array(['0004', '0005', '0006', ..., '2647', '2649', '2650'], dtype=object)

In [18]:
#Resample Tiles
#tile_ids_train = random.sample(list(tile_ids_train), 200)
#create a file
#picklefile = open('tile_ids_train', 'wb')
#pickle the dataframe
#pickle.dump(tile_ids_train, picklefile)
#close file
#picklefile.close()

#read the pickle file
picklefile = open('tile_ids_train', 'rb')
#unpickle the dataframe
tile_ids_train = pickle.load(picklefile)
#close file
picklefile.close()

#print the dataframe
tile_ids_train

['0829',
 '0758',
 '1237',
 '1091',
 '2412',
 '2142',
 '0053',
 '2259',
 '1274',
 '0558',
 '1443',
 '1621',
 '0086',
 '0907',
 '1756',
 '2402',
 '0858',
 '0881',
 '0427',
 '0660',
 '0491',
 '0904',
 '0697',
 '2637',
 '2561',
 '2407',
 '2647',
 '1550',
 '0876',
 '0738',
 '2236',
 '1296',
 '1781',
 '1351',
 '1816',
 '2149',
 '0388',
 '2274',
 '2311',
 '1516',
 '0527',
 '2387',
 '2352',
 '1853',
 '1141',
 '2417',
 '1476',
 '0153',
 '1201',
 '0466',
 '0806',
 '0987',
 '1791',
 '1127',
 '0310',
 '1862',
 '1601',
 '1792',
 '2507',
 '1246',
 '1475',
 '2526',
 '1985',
 '1361',
 '1077',
 '2524',
 '2293',
 '2467',
 '1198',
 '1572',
 '1464',
 '0634',
 '2525',
 '1889',
 '2012',
 '2020',
 '2298',
 '0602',
 '1003',
 '0324',
 '2441',
 '2169',
 '2368',
 '0244',
 '2575',
 '1758',
 '2420',
 '1667',
 '1492',
 '0611',
 '1720',
 '0724',
 '0671',
 '0492',
 '1210',
 '2632',
 '0936',
 '0846',
 '1400',
 '0836',
 '0861',
 '1084',
 '1637',
 '0205',
 '2017',
 '2297',
 '2608',
 '1605',
 '0423',
 '1494',
 '1586',
 

In [19]:
#X = np.empty((0, 2 * (n_obs-1)))
X = np.empty((0, 494)) #13 bandas * 38 datetime
y = np.empty((0, 1))
field_ids = np.empty((0, 1))
list_discarded=[]
for tile_id in tile_ids_train[175:200]:#tile_ids_train:
    
    if tile_id != '1951': # avoid using this specific tile for the Hackathon as it might have a missing file
        #print(tile_id)
        tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]
        
        label_src = rasterio.open(tile_df[tile_df['asset']=='labels']['file_path'].values[0])
        label_array = label_src.read(1)
        y = np.append(y, label_array.flatten())

        field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
        field_id_array = field_id_src.read(1)
        field_ids = np.append(field_ids, field_id_array.flatten())

        tile_date_times = tile_df[tile_df['satellite_platform']=='s2']['datetime'].unique()
        
        X_tile = np.empty((256 * 256, 0))
        tile_df
        for date_time in tile_date_times: #[ :  4 * n_obs : n_obs]
            for band in bands:
                band_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']==band)]['file_path'].values[0])
                band_array = np.expand_dims(band_src.read(1).flatten(), axis=1)
                X_tile = np.append(X_tile, band_array, axis = 1)
                
        X = np.append(X, X_tile, axis=0)
        print(tile_id,X.shape[0],y.shape[0])
        #if X.shape[0]!=y.shape[0]:
        #    list_discarded.append(tile_id)

2278 65536 65536
2059 131072 131072
0305 196608 196608
0254 262144 262144
1472 327680 327680
0501 393216 393216
0571 458752 458752
0870 524288 524288
0174 589824 589824
0740 655360 655360
2241 720896 720896
2343 786432 786432
0223 851968 851968
1074 917504 917504
0897 983040 983040
2410 1048576 1048576
0004 1114112 1114112
1843 1179648 1179648
0311 1245184 1245184
0168 1310720 1310720
0874 1376256 1376256
1668 1441792 1441792
1710 1507328 1507328
0557 1572864 1572864
1946 1638400 1638400


In [20]:
 tile_date_times

array(['2017-04-01T00:00:00+0000', '2017-04-11T00:00:00+0000',
       '2017-04-21T00:00:00+0000', '2017-05-01T00:00:00+0000',
       '2017-05-11T00:00:00+0000', '2017-05-21T00:00:00+0000',
       '2017-05-31T00:00:00+0000', '2017-06-10T00:00:00+0000',
       '2017-06-20T00:00:00+0000', '2017-06-30T00:00:00+0000',
       '2017-07-05T00:00:00+0000', '2017-07-10T00:00:00+0000',
       '2017-07-15T00:00:00+0000', '2017-07-20T00:00:00+0000',
       '2017-07-25T00:00:00+0000', '2017-07-30T00:00:00+0000',
       '2017-08-04T00:00:00+0000', '2017-08-09T00:00:00+0000',
       '2017-08-14T00:00:00+0000', '2017-08-19T00:00:00+0000',
       '2017-08-24T00:00:00+0000', '2017-08-29T00:00:00+0000',
       '2017-09-08T00:00:00+0000', '2017-09-18T00:00:00+0000',
       '2017-09-23T00:00:00+0000', '2017-09-28T00:00:00+0000',
       '2017-10-03T00:00:00+0000', '2017-10-08T00:00:00+0000',
       '2017-10-13T00:00:00+0000', '2017-10-18T00:00:00+0000',
       '2017-10-23T00:00:00+0000', '2017-10-28T00:00:00

In [21]:
print(X_tile.shape)
print(X.shape)
print(y.shape)
print(field_ids.shape)

(65536, 494)
(1638400, 494)
(1638400,)
(1638400,)


In [22]:
data = pd.DataFrame(X)
data['label'] = y.astype(int)
data['field_id'] = field_ids
data = data[data.field_id != 0]
data

,0,1,2,3,4,5,6,7,8,9,...,486,487,488,489,490,491,492,493,label,field_id
0,27.0,30.0,44.0,61.0,74.0,82.0,89.0,92.0,78.0,91.0,...,61.0,64.0,67.0,74.0,116.0,87.0,70.0,0.0,7,50057.0
1,27.0,32.0,46.0,63.0,74.0,82.0,89.0,94.0,78.0,91.0,...,61.0,64.0,69.0,74.0,116.0,87.0,70.0,0.0,7,50057.0
2,27.0,35.0,49.0,69.0,79.0,85.0,92.0,95.0,78.0,94.0,...,65.0,67.0,70.0,74.0,117.0,88.0,73.0,0.0,7,50057.0
3,27.0,38.0,53.0,74.0,79.0,85.0,92.0,98.0,78.0,94.0,...,65.0,67.0,72.0,74.0,117.0,88.0,73.0,0.0,7,50057.0
4,27.0,38.0,52.0,72.0,78.0,85.0,90.0,99.0,78.0,92.0,...,65.0,69.0,71.0,74.0,117.0,88.0,74.0,0.0,7,50057.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638395,56.0,51.0,58.0,71.0,80.0,81.0,83.0,86.0,138.0,80.0,...,69.0,72.0,77.0,78.0,114.0,97.0,76.0,0.0,1,52255.0
1638396,55.0,51.0,58.0,73.0,80.0,82.0,84.0,86.0,134.0,81.0,...,69.0,72.0,77.0,80.0,116.0,99.0,77.0,0.0,1,52255.0
1638397,55.0,51.0,59.0,73.0,80.0,82.0,84.0,87.0,134.0,81.0,...,69.0,72.0,77.0,80.0,116.0,99.0,77.0,0.0,1,52255.0
1638398,55.0,52.0,60.0,72.0,80.0,82.0,85.0,88.0,134.0,81.0,...,70.0,73.0,78.0,80.0,119.0,104.0,79.0,0.0,1,52255.0


In [23]:
data['label'].value_counts()

7    237881
2     93921
6     60948
5     54803
1     45571
8     44241
3     30990
9     26657
4     25420
Name: label, dtype: int64

In [24]:
#create a file
picklefile = open('data_train_df_175_to_200', 'wb')
#pickle the dataframe
pickle.dump(data, picklefile)
#close file
picklefile.close()

In [25]:
#read the pickle file
picklefile = open('data_train_df_175_to_200', 'rb')
#unpickle the dataframe
data_train_df = pickle.load(picklefile)
#close file
picklefile.close()

#print the dataframe
data_train_df

,0,1,2,3,4,5,6,7,8,9,...,486,487,488,489,490,491,492,493,label,field_id
0,27.0,30.0,44.0,61.0,74.0,82.0,89.0,92.0,78.0,91.0,...,61.0,64.0,67.0,74.0,116.0,87.0,70.0,0.0,7,50057.0
1,27.0,32.0,46.0,63.0,74.0,82.0,89.0,94.0,78.0,91.0,...,61.0,64.0,69.0,74.0,116.0,87.0,70.0,0.0,7,50057.0
2,27.0,35.0,49.0,69.0,79.0,85.0,92.0,95.0,78.0,94.0,...,65.0,67.0,70.0,74.0,117.0,88.0,73.0,0.0,7,50057.0
3,27.0,38.0,53.0,74.0,79.0,85.0,92.0,98.0,78.0,94.0,...,65.0,67.0,72.0,74.0,117.0,88.0,73.0,0.0,7,50057.0
4,27.0,38.0,52.0,72.0,78.0,85.0,90.0,99.0,78.0,92.0,...,65.0,69.0,71.0,74.0,117.0,88.0,74.0,0.0,7,50057.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638395,56.0,51.0,58.0,71.0,80.0,81.0,83.0,86.0,138.0,80.0,...,69.0,72.0,77.0,78.0,114.0,97.0,76.0,0.0,1,52255.0
1638396,55.0,51.0,58.0,73.0,80.0,82.0,84.0,86.0,134.0,81.0,...,69.0,72.0,77.0,80.0,116.0,99.0,77.0,0.0,1,52255.0
1638397,55.0,51.0,59.0,73.0,80.0,82.0,84.0,87.0,134.0,81.0,...,69.0,72.0,77.0,80.0,116.0,99.0,77.0,0.0,1,52255.0
1638398,55.0,52.0,60.0,72.0,80.0,82.0,85.0,88.0,134.0,81.0,...,70.0,73.0,78.0,80.0,119.0,104.0,79.0,0.0,1,52255.0


In [26]:
data_train_df = data_train_df.groupby('field_id').mean().reset_index()
data_train_df

,field_id,0,1,2,3,4,5,6,7,8,...,485,486,487,488,489,490,491,492,493,label
0,99.0,20.517045,21.553977,32.088068,41.372159,52.355114,63.036932,69.500000,75.792614,80.644886,...,53.477273,79.025568,88.588068,97.607955,95.176136,103.687500,75.977273,96.741477,0.0,4.0
1,164.0,23.166667,28.123188,34.152174,41.702899,49.123188,57.159420,63.224638,70.130435,77.340580,...,41.471014,55.898551,61.768116,66.797101,70.818841,85.659420,75.036232,68.528986,0.0,3.0
2,187.0,24.758929,30.348214,45.619048,61.723214,69.589286,75.607143,81.160714,87.047619,75.812500,...,64.041667,70.434524,74.571429,80.773810,81.047619,123.589286,102.312500,81.535714,0.0,2.0
3,360.0,47.908497,48.045752,56.555556,67.947712,76.862745,81.705882,86.143791,91.169935,130.790850,...,69.366013,75.509804,79.973856,85.633987,86.385621,111.738562,87.627451,85.555556,0.0,7.0
4,751.0,18.000000,22.270833,33.718750,46.197917,51.041667,53.687500,57.020833,61.260417,55.906250,...,75.614583,78.447917,82.010417,86.520833,87.927083,121.375000,97.437500,86.979167,0.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,120784.0,24.958449,31.385042,50.728532,67.628809,76.376731,80.268698,83.750693,86.684211,108.013850,...,81.795014,90.263158,95.288089,100.556787,95.897507,127.739612,110.177285,99.357341,0.0,9.0
822,121313.0,75.320615,70.493538,81.742769,95.460923,106.342769,108.534154,111.665231,115.495385,198.638154,...,42.150769,47.976615,53.001231,58.313846,62.171692,97.795077,70.720615,60.446154,0.0,2.0
823,121808.0,20.480519,22.298701,35.584416,48.584416,57.454545,63.116883,69.012987,73.324675,76.909091,...,54.363636,60.701299,64.818182,68.636364,71.714286,109.246753,89.532468,70.909091,0.0,5.0
824,122012.0,23.962366,29.053763,38.338710,48.155914,57.997312,67.475806,73.443548,79.298387,81.505376,...,38.405914,58.741935,66.045699,71.236559,73.978495,85.137097,62.795699,73.268817,0.0,4.0
